# Load Data

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

import joblib

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
plt.style.use('seaborn')

In [ ]:
sale_data = pd.read_csv('data_available/sale_data_post_out.csv', sep=',')
sale_data.shape  #### cambiar esto

In [ ]:
sale_data.info()

In [ ]:
features = [#'price',
            'floor_area',
            'views',
            'latitude',
            'longitude',
      #      'bedroom',
            'bathroom',
            #'sale_type',
            'type_house',
#            'postcode',
 #           'state_district',
  #          'county',
  #          'city_district',
   #         'road',
      #      'place',
            'code',
  #          'admin1',
  #          'cities'
]

#sale_data = sale_data.dropna().copy()
data = sale_data[features].copy()
data.shape

In [ ]:
data.isna().sum()

In [ ]:
num_features = data.select_dtypes('number').columns
cat_features =  data.select_dtypes('object').columns

-----------

# Split Data

In [ ]:
y = sale_data['price'].copy()
y.shape

In [ ]:
print(data.shape)
#X = data.copy()
X = pd.get_dummies(data).copy() #[num_features]
print(X.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.15)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

# Train and Metrics

En estadística, el **coeficiente de determinación**, denominado R² y pronunciado R cuadrado, es un estadístico usado en el contexto de un modelo estadístico cuyo principal propósito es predecir futuros resultados o probar una hipótesis. El coeficiente determina la calidad del modelo para replicar los resultados, y la proporción de variación de los resultados que puede explicarse por el modelo.

Es el porcentaje de la variación en la variable de respuesta que es explicado por un modelo lineal. Es decir:

R-cuadrado = Variación explicada / variación total

El R-cuadrado siempre está entre 0 y 100%:

In [ ]:
from sklearn import metrics
#metrics.SCORERS

In [ ]:
def metrics_regression(y_test, y_pred, squared=False):
    
    r2_score = metrics.r2_score(y_test, y_pred)
    mae = metrics.mean_absolute_error(y_test, y_pred)
    mape = metrics.mean_absolute_percentage_error(y_test, y_pred)
    mse = metrics.mean_squared_error(y_test, y_pred)
    rmse = metrics.mean_squared_error(y_test, y_pred, squared=squared)
    
    print(f'R²: {r2_score}')
    print(f'MAE: {mae}')
    print(f'MAPE: {mape}')
    #print(f'MSE: {mse}')
    print(f'RMSE: {rmse}')
    

https://scikit-learn.org/stable/modules/cross_validation.html

In [ ]:
from sklearn.model_selection import cross_validate

def cross_validate_custom(estimator, scoring_dict, X_train, y_train, cv=10, return_train_score=False):
    estimator = estimator
    scoring_dict = scoring_dict

    scores = cross_validate(estimator, X=X_train, y=y_train, scoring=scoring_dict, cv=cv, 
                            return_train_score=return_train_score)
    print(scores.keys())
    return scores

In [ ]:
def scores_statistics(estimator, scoring_dict, X_train, y_train, cv=10, return_train_score=False, time_info=False):
    
    scores = cross_validate(estimator, 
                            X=X_train, y=y_train, 
                            scoring=scoring_dict, 
                            cv=cv, 
                            return_train_score=return_train_score)
    
    if time_info:
        fit_time_mean = np.mean(scores['fit_time'])
        fit_time_std = np.std(scores['fit_time'])
        score_time_mean = np.mean(scores['score_time'])
        score_time_std = np.std(scores['score_time'])
        #time_list = []    
        print('fit_time mean:', fit_time_mean)
        print('fit_time std:', fit_time_std)
        print('score_time mean:', score_time_mea)
        print('score_time std:', score_time_std)
    
    
    for key in scoring_dict:
        try:
            mean = np.mean(scores['test_' + key])
            std = np.std(scores['test_' + key])
            print(key, 'mean:', mean)
            print(key, 'std:', std, '\n')
        except:
            continue
    return scores

-------------

------------

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
print(X_train.shape)
#X_train.isna().sum()

In [ ]:
# Get dummy data
X_train = pd.get_dummies(X_train).copy()
print(X_train.shape)
# esto me esta dando resultados muy malos, por que? -> ya no
#from sklearn.preprocessing import LabelEncoder
#
#le = LabelEncoder()
#
#for column in cat_features:
 #   X_train[column] = le.fit_transform(X_train[column])

In [ ]:
#print(X_train.shape)
#X_train.isna().sum()

In [ ]:
from sklearn import metrics

scoring = {'r2': 'r2', 
           'MAE': 'neg_mean_absolute_error', 
           'MAPE': 'neg_mean_absolute_percentage_error', 
           'RMSE': 'neg_root_mean_squared_error'}
print(X_train.shape)
scores_statistics(estimator=LinearRegression(), 
                  scoring_dict=scoring, 
                  X_train=X_train, 
                  y_train=y_train, 
                  cv=10, 
                  return_train_score=False)

In [ ]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression().fit(X_train, y_train)
#lr.intercept_, lr.coef_
#lr.score(X_train, y_train)

#X_test = pd.get_dummies(X_test).copy()

metrics_regression(y_test=y_test, 
                   y_pred=lr.predict(X_test), 
                   squared=False)

In [ ]:
joblib.dump(lr, 'models/linear_regression_01-11-2021.plk')

---------------

-------------------

# Polynomial Regression

In [246]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (StandardScaler, OneHotEncoder, 
                                   PolynomialFeatures)
from sklearn.compose import ColumnTransformer
from sklearn import metrics
from sklearn.impute import SimpleImputer

In [247]:
def split_train_test(df, test_ratio=.15):
    shuffled_indices = np.random.permutation(len(df))
    test_set_size = int(len(df) * test_ratio)
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    return df.iloc[train_indices], df.iloc[test_indices]

In [248]:
def split_x_y(train_set, test_set, features, target='price'):
    y_train = train_set[target].copy()
    X_train = train_set[features].copy()
    y_test = test_set[target].copy()
    X_test = test_set[features].copy()
    print('X_train:', X_train.shape, '\n' + 
          'X_test:', X_test.shape, '\n' + 
          'y_train:', y_train.shape, '\n' + 
          'y_test:', y_test.shape, '\n')
    return X_train, X_test, y_train, y_test

In [249]:
def scores_statistics(estimator, scoring_dict, X_train, y_train, cv=10, return_train_score=False, time_info=False):
    
    scores = cross_validate(estimator, 
                            X=X_train, y=y_train, 
                            scoring=scoring_dict, 
                            cv=cv, 
                            return_train_score=return_train_score)
    
    if time_info:
        fit_time_mean = np.mean(scores['fit_time'])
        fit_time_std = np.std(scores['fit_time'])
        score_time_mean = np.mean(scores['score_time'])
        score_time_std = np.std(scores['score_time'])
        #time_list = []    
        print('fit_time mean:', fit_time_mean)
        print('fit_time std:', fit_time_std)
        print('score_time mean:', score_time_mea)
        print('score_time std:', score_time_std)
    
    
    for key in scoring_dict:
        try:
            mean = np.mean(scores['test_' + key])
            std = np.std(scores['test_' + key])
            print(key, 'mean:', mean)
            print(key, 'std:', std, '\n')
        except:
            continue
    return scores

In [250]:
features = ['price',
            'floor_area',
            'views',
            'latitude',
            'longitude',
      #      'bedroom',
            'bathroom',
            #'sale_type',
            'type_house',
#            'postcode',
 #           'state_district',
  #          'county',
  #          'city_district',
   #         'road',
      #      'place',
            'code',
  #          'admin1',
  #          'cities'
           ]

In [251]:
sale_data = pd.read_csv('data_available/sale_data_post_out.csv', sep=',')
print(sale_data.shape)  #### cambiar esto

data = sale_data[features].copy()
print(data.shape)

(7387, 34)
(7387, 8)


In [252]:
print(data.isna().sum())

price            0
floor_area       0
views            0
latitude         0
longitude        0
bathroom         0
type_house       0
code          1328
dtype: int64


In [253]:
train_set, test_set = split_train_test(data, .15)
print(train_set.shape, test_set.shape)
#print(train_set.isna().sum())

(6279, 8) (1108, 8)


In [254]:
features = list(train_set.columns)
features.remove('price')
features

['floor_area',
 'views',
 'latitude',
 'longitude',
 'bathroom',
 'type_house',
 'code']

In [263]:
X_train, X_test, y_train, y_test = split_x_y(train_set=train_set, 
                                             test_set=test_set, 
                                             features=features, 
                                             target='price')

#print(X_train.isna().sum())

X_train: (6279, 7) 
X_test: (1108, 7) 
y_train: (6279,) 
y_test: (1108,) 



In [287]:
num_pipe = Pipeline([
    ('std_scaler', StandardScaler()),
    ('poly', PolynomialFeatures(degree=3, include_bias=False)),
    ])

#cat_features = train_set.select_dtypes('object').columns
type_house_levels = train_set.type_house.unique()
code_levels = train_set.code.unique()

cat_pipe = Pipeline([
    ('one_hot_encoder', OneHotEncoder(categories=[type_house_levels, code_levels]))  #code_levels
                                      #handle_unknown='ignore'
    ])

#num_features = data.select_dtypes('number').columns
#cat_features =  data.select_dtypes('object').columns
num_features = ['floor_area', 
                'views', 
                'latitude', 
                'longitude', 
                'bathroom']
cat_features = ['type_house',
                'code']

preprocessor = ColumnTransformer([
    ('num', num_pipe, num_features),
    ('cat', cat_pipe, cat_features),
    ]) #, remainder='passthrough'
#preprocessor


estimator = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('imputer', SimpleImputer(strategy='constant', 
                              fill_value=None)),
    ('regressor', LinearRegression())
                           ])
#estimator

In [264]:
scoring = {'r2': 'r2', 
           'MAE': 'neg_mean_absolute_error', 
           'MAPE': 'neg_mean_absolute_percentage_error', 
           'RMSE': 'neg_root_mean_squared_error'}
print(X_train.shape)

scores = scores_statistics(estimator=estimator, 
                           scoring_dict=scoring, 
                           X_train=X_train, 
                           y_train=y_train, 
                           cv=10, 
                           return_train_score=False)

(6279, 7)
r2 mean: 0.7409140436903824
r2 std: 0.04372488177613465 

MAE mean: -95497.44682321073
MAE std: 3504.089447897079 

MAPE mean: -0.2746810244943
MAPE std: 0.013699581137712797 

RMSE mean: -153433.27998466874
RMSE std: 9163.199683532202 



In [282]:
#%%time

from sklearn.model_selection import GridSearchCV, ParameterGrid

#param_grid = [
#    {'degree'=[1,2,3,4,5,6,7,8,9,0]},
#]


grid_search = GridSearchCV(estimator=estimator, 
                           param_grid={'preprocessor__num__poly__degree': [1,2,3,4,5]}, 
                           cv=5, 
                           n_jobs=-1,
                        #   scoring='neg_mean_squared_error',
                        #   return_train_score=True
                          )

grid_search.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('std_scaler',
                                                                                          StandardScaler()),
                                                                                         ('poly',
                                                                                          PolynomialFeatures(degree=3,
                                                                                                             include_bias=False))]),
                                                                         ['floor_area',
                                                                          'views',
                                                                          'latitude',
                         

In [283]:
grid_search.best_params_

{'preprocessor__num__poly__degree': 3}

In [284]:
grid_search.best_score_

0.7442833804212804

In [285]:
#grid_search.best_estimator_

In [286]:
pd.DataFrame(grid_search.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_preprocessor__num__poly__degree,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.125092,0.010746,0.011654,0.000690,1,{'preprocessor__num__poly__degree': 1},0.641357,0.668933,0.669079,0.682320,0.639839,0.660306,0.016817,4
1,0.262651,0.024497,0.016081,0.001651,2,{'preprocessor__num__poly__degree': 2},0.699572,0.743389,0.754738,0.710256,0.710744,0.723740,0.021362,2
2,0.934982,0.023024,0.009851,0.000494,3,{'preprocessor__num__poly__degree': 3},0.706135,0.758940,0.779665,0.734083,0.742594,0.744283,0.024612,1
3,0.297836,0.051353,0.022748,0.005438,4,{'preprocessor__num__poly__degree': 4},0.652915,0.778908,0.717026,0.728861,0.736422,0.722826,0.040713,3
4,0.400963,0.063450,0.018939,0.003567,5,{'preprocessor__num__poly__degree': 5},-1.915671,0.642585,0.553581,-1.175276,-0.064963,-0.391949,1.000654,5


# Decission Tree Regressor

In [288]:
from sklearn.tree import DecisionTreeRegressor

In [289]:
num_pipe = Pipeline([
    ('std_scaler', StandardScaler()),
#    ('poly', PolynomialFeatures(degree=3, include_bias=False)),
    ])

#cat_features = train_set.select_dtypes('object').columns
type_house_levels = train_set.type_house.unique()
code_levels = train_set.code.unique()

cat_pipe = Pipeline([
    ('one_hot_encoder', OneHotEncoder(categories=[type_house_levels, code_levels]))  #code_levels
                                      #handle_unknown='ignore'
    ])

#num_features = data.select_dtypes('number').columns
#cat_features =  data.select_dtypes('object').columns
num_features = ['floor_area', 
                'views', 
                'latitude', 
                'longitude', 
                'bathroom']
cat_features = ['type_house',
                'code']

preprocessor = ColumnTransformer([
    ('num', num_pipe, num_features),
    ('cat', cat_pipe, cat_features),
    ]) #, remainder='passthrough'
#preprocessor


estimator = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('imputer', SimpleImputer(strategy='constant', 
                              fill_value=None)),
    ('regressor', DecisionTreeRegressor())
                           ])
#estimator

In [300]:
grid_search = GridSearchCV(estimator=estimator, 
                           param_grid={
                        #       'preprocessor__num__poly__degree': [1,2,3,4,5],
                               'regressor__max_depth': range(1, 15), #[2,3,4,5,6,7]
                           }, 
                           cv=5, 
                           n_jobs=-1,
                        #   scoring='neg_mean_squared_error',
                        #   return_train_score=True
                          )

grid_search.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('std_scaler',
                                                                                          StandardScaler())]),
                                                                         ['floor_area',
                                                                          'views',
                                                                          'latitude',
                                                                          'longitude',
                                                                          'bathroom']),
                                                                        ('cat',
                                                                         Pipeline(steps=[('one_hot_encoder',
         

In [306]:
print(grid_search.best_params_)
print(grid_search.best_score_)

{'regressor__max_depth': 6}
0.6972255986935948


In [304]:
#pd.DataFrame(grid_search.cv_results_)

# Random Forest Regressor

In [307]:
from sklearn.ensemble import RandomForestRegressor

```python
RandomForestRegressor(n_estimators=100, 
                      *, 
                      criterion='squared_error', 
                      max_depth=None, 
                      min_samples_split=2, 
                      min_samples_leaf=1, 
                      min_weight_fraction_leaf=0.0, 
                      max_features='auto', 
                      max_leaf_nodes=None, 
                      min_impurity_decrease=0.0, 
                      bootstrap=True, 
                      oob_score=False, 
                      n_jobs=None, 
                      random_state=None, 
                      verbose=0, 
                      warm_start=False, 
                      ccp_alpha=0.0, 
                      max_samples=None)
```

In [319]:
rfr_pipe_est = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('imputer', SimpleImputer(strategy='constant', 
                              fill_value=None)),
    ('regressor', RandomForestRegressor())
                           ])
#estimator

In [328]:
param_grid = [
    {'regressor__n_estimators': [3,10,30], 
     'regressor__max_features': range(1, 8)},
    {'regressor__n_estimators': range(41,60)}, #[3,10,30,40,50,60]
]

grid_search = GridSearchCV(estimator=rfr_pipe_est, 
                           param_grid=param_grid,
                           #{
                        #       'preprocessor__num__poly__degree': [1,2,3,4,5],
                        #       'regressor__max_depth': range(1, 15), #[2,3,4,5,6,7]
                        #   }, 
                           cv=5, 
                           n_jobs=-1,
                        #   scoring='neg_mean_squared_error',
                        #   return_train_score=True
                          )

grid_search.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('std_scaler',
                                                                                          StandardScaler())]),
                                                                         ['floor_area',
                                                                          'views',
                                                                          'latitude',
                                                                          'longitude',
                                                                          'bathroom']),
                                                                        ('cat',
                                                                         Pipeline(steps=[('one_hot_encoder',
         

In [329]:
print(grid_search.best_params_)
print(grid_search.best_score_)

{'regressor__n_estimators': 44}
0.7998526547606316


In [326]:
pd.DataFrame(grid_search.cv_results_).tail()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_regressor__max_features,param_regressor__n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
35,6.685643,0.013066,0.031862,0.000349,NaN,55,{'regressor__n_estimators': 55},0.800315,0.786868,0.833414,0.758655,0.799107,0.795672,0.024106,14
36,6.954544,0.127430,0.039644,0.004769,NaN,56,{'regressor__n_estimators': 56},0.799614,0.783382,0.838665,0.767620,0.793577,0.796572,0.023674,8
37,7.232374,0.050847,0.033035,0.000805,NaN,57,{'regressor__n_estimators': 57},0.799052,0.788281,0.838785,0.757896,0.797805,0.796364,0.025900,9
38,6.659633,0.364550,0.023407,0.000420,NaN,58,{'regressor__n_estimators': 58},0.800109,0.787244,0.840482,0.758941,0.803283,0.798012,0.026376,3
39,6.372685,0.248393,0.021708,0.001232,NaN,59,{'regressor__n_estimators': 59},0.805372,0.786744,0.842662,0.757457,0.793277,0.797102,0.027705,7


In [330]:
rfr_pipe_est = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('imputer', SimpleImputer(strategy='constant', 
                              fill_value=None)),
    ('regressor', RandomForestRegressor(n_estimators=44))
                           ])
#estimator

scores = scores_statistics(estimator=rfr_pipe_est, 
                           scoring_dict=scoring, 
                           X_train=X_train, 
                           y_train=y_train, 
                           cv=10, 
                           return_train_score=False)

r2 mean: 0.8002624958393272
r2 std: 0.026967885518696666 

MAE mean: -77849.77205209687
MAE std: 3186.794880910816 

MAPE mean: -0.21704229815241996
MAPE std: 0.008744707519253879 

RMSE mean: -135114.36764980553
RMSE std: 10821.74195684397 

